<a href="https://colab.research.google.com/github/jinhyukbae/playdata/blob/main/%EC%98%81%ED%99%94%EA%B0%90%EC%83%81%ED%8F%89%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D%ED%95%9C%EA%B8%80%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
# https://github.com/e9t/nsmc/

# 네이버 영화 리뷰에 대한 감성분석 긍정/부정 관련된 분석
file_url = 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt'
df = pd.read_csv(file_url, sep='\t', index_col=0)
df.head() # 20만개 데이터

,document,label
id,,
8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [ ]:
len(df)

200000

In [ ]:
df.info() #결측치 8개 드롭 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 8112052 to 8548411
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   document  199992 non-null  object
 1   label     200000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 4.6+ MB


In [ ]:
df2 = df.dropna() #결측치 모두 삭제
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199992 entries, 8112052 to 8548411
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   document  199992 non-null  object
 1   label     199992 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 4.6+ MB


In [ ]:
# 같은 모양의 df를 연결할 때 pd.concat
df3 = pd.concat([
    df2.head(1000), # 행 단위로 합치는 거 기때문에 axis=1 안해도 됨
    df2.tail(1000)
])
# 위 아래로 천개씩 토탈 2천개로만 해보자
df3

,document,label
id,,
8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...
8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
3302770,쓰.레.기,0
5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0


## 한글 전처리

In [ ]:
df3.document.iloc[0]
# 어리다 -> 어릴 / 재미+나다 -> 구성이 복잡해~
# 한글 자연어 분석 -> 형태소(언어의 최소 의미 단위)로 분석 해야함
# 형태소 분석기 사용 

# 띄어쓰기가 제대로 안돼있음

'어릴때보고 지금다시봐도 재밌어요ㅋㅋ'

In [ ]:
# https://konlpy.org/ko/latest/index.html
!pip install konlpy --quiet

     |████████████████████████████████| 19.4 MB 8.1 MB/s 
     |████████████████████████████████| 465 kB 59.3 MB/s 


In [ ]:
from konlpy.tag import Okt

In [ ]:
okt = Okt()

In [ ]:
sample_text = df3.document.iloc[0]
sample_text

'어릴때보고 지금다시봐도 재밌어요ㅋㅋ'

In [ ]:
okt.nouns(sample_text) #명사 추출하는 nouns

['때', '보고', '지금', '다시']

In [ ]:
df3['nouns'] = df3.document.apply(okt.nouns).apply(
    # nouns = okt.nouns로 분석된 명사들 리스트
    lambda nouns: [n for n in nouns if len(n) >= 2])
# 2글자 미만의 명사들은 모두 제외
# 명사로 분석된 nouns를 n에 넣고 len(n)의 길이가 2개 이상인 것만 남기겠다
df3['nouns']

id
8112052                                          [보고, 지금, 다시]
8132799     [디자인, 학생, 외국, 디자이너, 일군, 전통, 통해, 발전, 문화, 산업, 사실...
4655635                             [폴리스스토리, 시리즈, 부터, 하나, 최고]
9251303                              [연기, 진짜, 생각, 몰입, 진짜, 영화]
10067386                                   [안개, 밤하늘, 초승달, 영화]
                                  ...                        
8963373                                             [포켓, 몬스터]
3302770                                                    []
5458175                       [완전, 사이코, 영화, 마지막, 더욱더, 영화, 린다]
6908648                                       [라따뚜이, 스머프, 런가]
8548411                                      [저그, 영차, 영차, 영차]
Name: nouns, Length: 2000, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
# cv.fit_transform(df3.nouns) # 리스트X 문자열로 넣어줘야함
x = cv.fit_transform(df3.nouns.apply(" ".join))
print(x)

  (0, 1206)	1
  (0, 2670)	1
  (0, 551)	1
  (1, 722)	1
  (1, 3120)	1
  (1, 2059)	1
  (1, 721)	1
  (1, 2309)	1
  (1, 2479)	2
  (1, 2972)	1
  (1, 1119)	1
  (1, 1011)	1
  (1, 1363)	1
  (1, 1345)	1
  (1, 2090)	1
  (1, 1640)	1
  (1, 1964)	1
  (1, 494)	1
  (1, 1333)	1
  (2, 3067)	1
  (2, 1628)	1
  (2, 1266)	1
  (2, 3098)	1
  (2, 2814)	1
  (3, 1945)	1
  :	:
  (1990, 2892)	1
  (1990, 859)	1
  (1990, 629)	1
  (1991, 1977)	1
  (1991, 1957)	1
  (1991, 67)	1
  (1991, 2347)	1
  (1991, 435)	1
  (1992, 2947)	1
  (1993, 1990)	1
  (1994, 102)	1
  (1994, 1202)	1
  (1995, 3057)	1
  (1995, 963)	1
  (1997, 1977)	2
  (1997, 2044)	1
  (1997, 847)	1
  (1997, 1353)	1
  (1997, 625)	1
  (1997, 819)	1
  (1998, 766)	1
  (1998, 1578)	1
  (1998, 746)	1
  (1999, 2447)	1
  (1999, 1976)	3


In [ ]:
cv.vocabulary_

In [ ]:
from sklearn.model_selection import train_test_split

y= df3.label # 타깃, 종속변수
x_train, x_test, y_train, y_test = train_test_split(x,y)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(x_train, y_train)
pred = model.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy_score(y_test, pred)

0.68

In [ ]:
https://colab.research.google.com/drive/1LPvMgmPFOfpWgQsck0t5yCIi97ncrM3f?usp=sharing